# Active inference - Chapter 2

## Posteiror probability
$$
P(x|y) = \frac{P(x)P(y|x)}{P(y)}
$$

In [2]:
import numpy as np

keys = ['frog', 'apple']
# index 0: frog, 1: apple
prior_beliefs = np.array([0.1, 0.9])

# 1st index 0: frog, 1: apple
# 2nd index 0: jumps, 1: stays
likelihoods = np.array([[0.81, 0.19],
                        [0.01, 0.99]])

# probability of jumping
p_jump = prior_beliefs @ likelihoods[:, 0]
print('p_jump: {:.3f}'.format(p_jump))
# posterior belief
posterior_beliefs = likelihoods[:, 0] * prior_beliefs / p_jump
for i in range(2):
    print('posterior belief of {}: {:.3f}'.format(keys[i], posterior_beliefs[i]))

p_jump: 0.090
posterior belief of frog: 0.900
posterior belief of apple: 0.100


## Define surprise
KL-Divergence can be defined as:
$$
D_{KL} \left[ Q(x) \| P(x) \right] = \mathbb{E}_{Q(x)} \left[ \text{ln } Q(x) - \text{ln } P(x) \right]
$$

Using KL-Divergence, we can quantify the Bayesian surprise in the example as:
$$
D_{KL} \left[ P(x|y) \| P(x) \right]
$$

In [3]:
def kl_divergence(p, q):
    return np.sum(p * np.log(p / q))

# KL divergence between prior and posterior
kl = kl_divergence(prior_beliefs, posterior_beliefs)
print('Surprise: {:.3f}'.format(kl))

KL divergence: 1.758


## Minimizing Variatonal Free Energy
The exact Bayesian inference supporting perception and action is computationally intractable: model evidence ($P(y)$) and the posterior probability ($P(x|y)$) cannot be computed for two possible reasons:

    1. For complex models, there may be many types of hidden states that all need marginalizing out (= making the probel computationally intractable)
    2. The marginalization operation might require analytically intractable integrals

Although posterior probability and model evidence are intractable quantaties, it can be computed efficiently with two approximated quantities: *approximate posterior* $Q$ and a *variational free energy*, $F$.

$$
F[Q, y] = -\mathbb{E}_{Q(x)} \left[ \text{ln } P(y, x) \right] - H \left[ Q(x) \right]\\
%= D_{KL} \left[ Q(x) \| P(x) \right] - \mathbb{E}_{Q(x)} \left[ \text{ln } P(y|x) \right] \\
%= D_{KL} \left[ Q(x) \| P(x|y) \right] - \text{ln } P(y)
$$